In [10]:
import pandas as pd
import utils

In [11]:
pair = "EUR_USD"
granularity = "H1"

In [12]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [13]:
non_cols = ['time', 'volume']

In [14]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [15]:
df.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [16]:
mod_cols

['mid_o',
 'mid_h',
 'mid_l',
 'mid_c',
 'bid_o',
 'bid_h',
 'bid_l',
 'bid_c',
 'ask_o',
 'ask_h',
 'ask_l',
 'ask_c']

In [18]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [20]:
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000
mean,5587.215054,1.025111,1.026129,1.024119,1.025119,1.025014,1.026042,1.024022,1.025032,1.025209,1.026225,1.024206,1.025207
std,3791.525597,0.032993,0.032904,0.033068,0.032998,0.032994,0.032904,0.033067,0.032998,0.032992,0.032904,0.033069,0.032998
min,25.000000,0.953960,0.955960,0.953600,0.953940,0.953880,0.955890,0.953520,0.953860,0.954030,0.956050,0.953680,0.954010
25%,3043.000000,0.998210,0.999325,0.997155,0.998235,0.998135,0.999245,0.997070,0.998155,0.998295,0.999410,0.997240,0.998315
50%,4823.000000,1.021930,1.022850,1.020880,1.021940,1.021800,1.022750,1.020750,1.021870,1.022090,1.023000,1.020960,1.022080
75%,7192.500000,1.053245,1.054020,1.052200,1.053240,1.053100,1.053945,1.052120,1.053160,1.053320,1.054115,1.052290,1.053325
max,35086.000000,1.092080,1.092960,1.091720,1.092070,1.091990,1.092880,1.091640,1.091990,1.092160,1.093040,1.091800,1.092150
